In [13]:
import detectevent

In [17]:
from detectevent import DetectEvent

ImportError: cannot import name 'DetectEvent' from 'detectevent' (/Users/jacquelinebuford/Documents/cee575/data-segmentation/detectevent/__init__.py)

In [18]:
detector = detectevent.DetectEvent()

AttributeError: module 'detectevent' has no attribute 'DetectEvent'

In [21]:
import sys
import os

# Add the directory containing your package to Python's path
# Replace with the actual path to the directory containing your package
package_parent_dir = "/detectevent/"
sys.path.append(package_parent_dir)

# Now try importing
from detectevent import DetectEvent

ImportError: cannot import name 'DetectEvent' from 'detectevent' (/Users/jacquelinebuford/Documents/cee575/data-segmentation/detectevent/__init__.py)

In [ ]:


# Load data - adjust the file path and column names to match your CSV
data = detector.load_data("usgs_gage_data.csv",
                            time_column="datetime",
                            value_column="gage_height",
                            date_format="%Y-%m-%d %H:%M:%S")

# Detect events using a simple threshold approach
events = detector.detect_events_threshold(threshold=10.0,  # adjust threshold based on your data
                                        min_duration=30,   # minimum 30 minutes
                                        rise_rate=0.5)     # 0.5 feet per hour rise rate

# Alternatively, detect events using moving average approach
# events = detector.detect_events_moving_average(window_size=48,  # 48 data points
#                                              std_factor=2.5,   # 2.5 standard deviations
#                                              min_duration=30)  # minimum 30 minutes

# Visualize the results
detector.visualize_events(events, title="USGS Gage Height Events")